In [ ]:
# %%
import sys, os

try:
    # ✅ Running from a Python script (.py file)
    base_path = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

SRC_PATH = os.path.join(base_path)

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

In [ ]:
# nodes/await_user_confirmation_node.py

from langgraph.types import Command
from utils.message_tagging import tag_message
from typing import Literal

def await_user_confirmation_node(state) -> Command[Literal["slot_collector", "intent_agent", "await_user_confirmation"]]:
    messages = state.get("messages", [])

    # Get latest user message
    latest_input = ""
    for msg in reversed(messages):
        if msg["role"] == "user":
            latest_input = msg["content"]
            break

    # Tag it
    user_tag = tag_message(latest_input)

    if user_tag == "confirmation":
        return Command(goto="intent_agent")

    elif user_tag == "denial":
        return Command(goto="slot_collector")

    else:
        # Not clear, ask again
        messages.append({
            "role": "assistant",
            "content": "Sorry, could you please confirm again with a clear yes or no?",
            "meta": "clarification_request"
        })

        return Command(
            update={"messages": messages},
            goto="await_user_confirmation"
        )
